In [ ]:
import pysam
import pandas as pd
import glob

In [ ]:
files = glob.glob("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231116pair_cl/*/*/matched_contig_annotations.csv")

In [ ]:
files

In [ ]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231116pair_cl/BT12K/05.match/matched_contig_annotations.csv")

In [ ]:
df

In [ ]:
df[["contig_id", "raw_clonotype_id"]].set_index("contig_id").to_dict(orient='dict')["raw_clonotype_id"]

In [ ]:
files = glob.glob("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231116pair_cl/*/*/matched_contig.fasta")

In [ ]:
files

In [ ]:
with pysam.FastxFile(files[0]) as f:
    for read in f:
        print(read.name)
        print(read.sequence)
        break

In [ ]:
for file in files:
    sample = file.split('/')[-3]
    out = f"/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120flseq_tsv/{sample}_flseq.tsv"
    
    s1, s2 = [], []
    with pysam.FastxFile(file) as f:
        for read in f:
            s1.append(read.name)
            s2.append(read.sequence)
    df = pd.DataFrame({'contig_id':s1, 'sequence':s2})
    df.to_csv(out, sep='\t', index=False)

In [277]:
clonotypes = glob.glob("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/*/04.summarize/clonotypes.csv")
annos = glob.glob("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/*/04.summarize/filtered_contig_annotations.csv")
fastas = glob.glob("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/*/04.summarize/filtered_contig.fasta")

In [278]:
clonotypes

['/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/BT12K-B/04.summarize/clonotypes.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B6K-B/04.summarize/clonotypes.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B12K-B/04.summarize/clonotypes.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/BT6K-B/04.summarize/clonotypes.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/Group2-B/04.summarize/clonotypes.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/Group1-B/04.summarize/clonotypes.csv']

In [279]:
annos

['/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/BT12K-B/04.summarize/filtered_contig_annotations.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B6K-B/04.summarize/filtered_contig_annotations.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B12K-B/04.summarize/filtered_contig_annotations.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/BT6K-B/04.summarize/filtered_contig_annotations.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/Group2-B/04.summarize/filtered_contig_annotations.csv',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/Group1-B/04.summarize/filtered_contig_annotations.csv']

In [280]:
fastas

['/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/BT12K-B/04.summarize/filtered_contig.fasta',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B6K-B/04.summarize/filtered_contig.fasta',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B12K-B/04.summarize/filtered_contig.fasta',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/BT6K-B/04.summarize/filtered_contig.fasta',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/Group2-B/04.summarize/filtered_contig.fasta',
 '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/Group1-B/04.summarize/filtered_contig.fasta']

In [ ]:
for i in range(len(clonotypes)):
    sample = clonotypes[i].split('/')[-3]
    
    cl = pd.read_csv(clonotypes[i])
    cl.cdr3s_aa = cl.cdr3s_aa.apply(lambda x: x.split(';'))
    cl = cl[cl['cdr3s_aa'].apply(lambda x: len(x)) == 2]
    cl = cl[cl['cdr3s_aa'].apply(lambda x: x[0]).str.contains('IGH')]
    cl = cl[~cl['cdr3s_aa'].apply(lambda x: x[1]).str.contains('IGH')]
    cl['cdr3s_aa'] = cl['cdr3s_aa'].apply(lambda x: ';'.join(x))
    
    anno = pd.read_csv(annos[i])
    anno = anno[['barcode' ,'contig_id', "raw_clonotype_id", "chain"]]
    
    s1, s2 = [], []
    with pysam.FastxFile(fastas[i]) as f:
        for read in f:
            s1.append(read.name)
            s2.append(read.sequence)
    fl_seq_dict = dict(zip(s1, s2))
    
    anno['fl_seq'] = anno['contig_id'].apply(lambda x: fl_seq_dict[x])
    anno['t'] = anno['raw_clonotype_id'].apply(lambda x: x[9:])
    anno['t'] = anno['t'].astype(int)
    anno = anno.sort_values(['t','barcode'])
    del anno['t']
    
    cl = cl[['clonotype_id', 'cdr3s_aa', 'cdr3s_nt']]
    anno = anno.rename(columns = {'raw_clonotype_id': 'clonotype_id'})
    df_merge = pd.merge(cl, anno, on='clonotype_id', how="inner")
    
    df_merge.to_csv(f"/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/{sample}.tsv", sep='\t', index=False)
    

In [ ]:
cl = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B12K-B/04.summarize/clonotypes.csv")

In [ ]:
#cl

In [ ]:
cl.cdr3s_aa = cl.cdr3s_aa.apply(lambda x: x.split(';'))
cl = cl[cl['cdr3s_aa'].apply(lambda x: len(x)) == 2]
cl = cl[cl['cdr3s_aa'].apply(lambda x: x[0]).str.contains('IGH')]
cl = cl[~cl['cdr3s_aa'].apply(lambda x: x[1]).str.contains('IGH')]
cl['cdr3s_aa'] = cl['cdr3s_aa'].apply(lambda x: ';'.join(x))

In [ ]:
cl

In [ ]:
anno = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B12K-B/04.summarize/filtered_contig_annotations.csv")

In [ ]:
#anno

In [ ]:
anno = anno[['barcode' ,'contig_id', "raw_clonotype_id", "chain"]]

In [ ]:
anno

In [ ]:
s1, s2 = [], []
with pysam.FastxFile("/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/B12K-B/04.summarize/filtered_contig.fasta") as f:
    for read in f:
        s1.append(read.name)
        s2.append(read.sequence)
fl_seq_dict = dict(zip(s1, s2))

In [ ]:
anno['fl_seq'] = anno['contig_id'].apply(lambda x: fl_seq_dict[x])

In [ ]:
anno

In [ ]:
anno['t'] = anno['raw_clonotype_id'].apply(lambda x: x[9:])

In [ ]:
anno['t'] = anno['t'].astype(int)

In [ ]:
anno = anno.sort_values(['t','barcode'])

In [ ]:
del anno['t']

In [ ]:
anno

In [ ]:
cl = cl[['clonotype_id', 'cdr3s_aa', 'cdr3s_nt']]

In [ ]:
cl

In [ ]:
anno = anno.rename(columns = {'raw_clonotype_id': 'clonotype_id'})

In [ ]:
df_merge = pd.merge(cl, anno, on='clonotype_id', how="inner")

In [ ]:
df_merge

In [ ]:
for i in range(len(clonotypes)):
    sample = clonotypes[i].split('/')[-3]
    
    cl = pd.read_csv(clonotypes[i])
    cl_raw = cl.copy()
    cl.cdr3s_aa = cl.cdr3s_aa.apply(lambda x: x.split(';'))
    cl = cl[cl['cdr3s_aa'].apply(lambda x: len(x)) == 2]
    cl = cl[cl['cdr3s_aa'].apply(lambda x: x[0]).str.contains('IGH')]
    cl = cl[~cl['cdr3s_aa'].apply(lambda x: x[1]).str.contains('IGH')]
    cl['cdr3s_aa'] = cl['cdr3s_aa'].apply(lambda x: ';'.join(x))
    pair = set(cl.clonotype_id)
    cl = cl_raw[~cl_raw['clonotype_id'].isin(pair)]
    
    anno = pd.read_csv(annos[i])
    anno = anno[['barcode' ,'contig_id', "raw_clonotype_id", "chain"]]
    
    s1, s2 = [], []
    with pysam.FastxFile(fastas[i]) as f:
        for read in f:
            s1.append(read.name)
            s2.append(read.sequence)
    fl_seq_dict = dict(zip(s1, s2))
    
    anno['fl_seq'] = anno['contig_id'].apply(lambda x: fl_seq_dict[x])
    anno['t'] = anno['raw_clonotype_id'].apply(lambda x: x[9:])
    anno['t'] = anno['t'].astype(int)
    anno = anno.sort_values(['t','barcode'])
    del anno['t']
    
    cl = cl[['clonotype_id', 'cdr3s_aa', 'cdr3s_nt']]
    anno = anno.rename(columns = {'raw_clonotype_id': 'clonotype_id'})
    df_merge = pd.merge(cl, anno, on='clonotype_id', how="inner")
    
    df_merge.to_csv(f"/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20231120/multi_{sample}.tsv", sep='\t', index=False)
    